<a href="https://colab.research.google.com/github/p-disha/ShopUNow-Agent/blob/main/ShopUNow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#########################################################################################


##Tech Stack

* LangChain / langchain_community-	Provides VectorStores (FAISS), Document abstraction, Embeddings, and Retrieval.


* FAISS (vectorstore)-	For embedding storage & similarity search (RAG).
Sentence-Transformers embeddings-	To convert document chunks into embedding vectors.


* **pdfminer.six + pytesseract + PIL**-	Extract text from PDFs, images (OCR) and markdown/text files — for building corpus.


* Markdownify	Convert markdown files to plain text.


* LangGraph (StateGraph etc.)-	The agent orchestration framework: state + nodes + transitions.


* Pydantic-	For structured schemas of state and tool inputs (validation, typing).


* LLM backends- OpenAI, Gemini (if available)	For synthesis / general LLM responses.

**Parses different document types (text, csv, pdf, image) into a corpus.**

**Chunks documents into manageable pieces using RecursiveCharacterTextSplitter.**

**Builds a FAISS index, persists it.**

**Sets up intent routing + tools for order status, returns, tickets.**

**Handles RAG retrieval + LLM synthesis with system prompt.**

**Passes retriever via RunnableConfig/configurable, avoiding earlier bug.**

**Good structure using StateGraph, Pydantic state schemas.**

In [17]:
# =========================
# Cell A – Setup & LLM with safer secret handling (Render-safe)
# =========================
import os
import subprocess
import sys
import threading

# Detect runtime
IS_RENDER = bool(os.getenv("PORT") or os.getenv("RENDER_SERVICE_TYPE"))

# ---------------------------------------------
# Dev-only dependency install (skip in Render)
# ---------------------------------------------
if not IS_RENDER:
    deps = [
        "langchain_community", "faiss-cpu", "langchain-openai",
        "langchain-google-genai", "pydantic", "typing_extensions",
        "vaderSentiment", "langgraph", "rapidfuzz", "flask",
        "flask-cors", "pyngrok"
    ]
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-qU", *deps], check=False)
    except Exception as ex:
        print("⚠️ Dev install failed:", ex, flush=True)

# ---------------------------------------------
# Secrets / API Keys
# ---------------------------------------------
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")

if not IS_RENDER:
    try:
        # Only attempt in Colab
        from google.colab import userdata  # type: ignore
        OPENAI_API_KEY = OPENAI_API_KEY or userdata.get("OPENAI_API_KEY")
        GEMINI_API_KEY = GEMINI_API_KEY or userdata.get("GEMINI_API_KEY")
        if OPENAI_API_KEY:
            os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
        if GEMINI_API_KEY:
            os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
    except Exception:
        pass  # ignore if not in Colab

if IS_RENDER:
    assert OPENAI_API_KEY or GEMINI_API_KEY, (
        "Production requires OPENAI_API_KEY or GOOGLE_API_KEY."
    )

# ---------------------------------------------
# Lazy Initialization Flags
# ---------------------------------------------
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
LLM = None
vector_store = None
faq_docs = None
_initialized = False
_init_lock = threading.Lock()

# ---------------------------------------------
# Lazy Init Function (defers FAISS & LLM setup)
# ---------------------------------------------
def initialize_backend():
    """Thread-safe one-time heavy setup for embeddings, FAISS, and LLM."""
    global _initialized, vector_store, faq_docs, LLM
    if _initialized:
        return True

    with _init_lock:
        if _initialized:
            return True
        try:
            print("⚙️ Initializing FAISS + LLM …", flush=True)

            # ---- Import heavy modules lazily ----
            from langchain_openai import OpenAIEmbeddings
            from langchain_core.documents import Document
            from langchain_community.vectorstores import FAISS
            from langchain_community.docstore.in_memory import InMemoryDocstore
            import faiss

            # ---- Tiny seed docs (safe for cold start) ----
            faq_docs = [
                Document(page_content="Support hours are 9 AM–9 PM IST, Monday–Saturday", metadata={"department": "Customer Support"}),
                Document(page_content="Return window is 10 days from delivery", metadata={"department": "Orders & Returns"}),
                Document(page_content="We accept UPI, credit cards, wallets & COD", metadata={"department": "Payments & Billing"})
            ]

            # ---- Embeddings + FAISS store ----
            embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)
            dim = len(embeddings.embed_query("hello world"))
            index = faiss.IndexFlatL2(dim)
            vector_store = FAISS(
                embedding_function=embeddings,
                index=index,
                docstore=InMemoryDocstore({}),
                index_to_docstore_id={}
            )
            vector_store.add_documents(faq_docs, ids=[f"doc{i}" for i in range(len(faq_docs))])

            # ---- Initialize LLM lazily ----
            LLM = get_chat_model()

            _initialized = True
            print("✅ Backend initialization complete.", flush=True)
            return True

        except Exception as e:
            print("❌ Backend init failed:", e, flush=True)
            return False

# ---------------------------------------------
# Chat Model Getter
# ---------------------------------------------
def get_chat_model():
    """Returns available LLM (Gemini > OpenAI > mock)."""
    if GEMINI_API_KEY:
        try:
            from langchain_google_genai import ChatGoogleGenerativeAI
            return ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)
        except Exception as e:
            print("Gemini init failed:", e, flush=True)

    if OPENAI_API_KEY:
        try:
            from langchain_openai import ChatOpenAI
            return ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
        except Exception as e:
            print("OpenAI init failed:", e, flush=True)

    # Fallback mock model
    from langchain_core.messages import HumanMessage
    class _Mock:
        def invoke(self, msgs):
            last = next((m.content for m in reversed(msgs) if isinstance(m, HumanMessage)), "")
            return type("Resp", (), {"content": "[MOCK] " + last})
    print("🧩 Using mock LLM fallback", flush=True)
    return _Mock()

SYSTEM_POLICY = (
    "You are ShopUNow Assistant. Be concise and accurate. "
    "Use internal knowledge base when possible. "
    "If unsure, ask a clarifying question."
)

print("✅ Cell A loaded — heavy init deferred until first use.")


✅ Cell A loaded — heavy init deferred until first use.


In [18]:
# =========================
# Cell A.1 – Lazy Load FAQ Dataset & Build Vector Store (Render-safe)
# =========================
import os
import json
import faiss
import threading
from typing import List, Tuple
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# --- Global cache + lock ---
_FAQ_VECTOR_STORE = None
_FAQ_DOCS: List[Document] = []
_FAQ_PATH = None
_FAQ_INIT_LOCK = threading.Lock()


def resolve_faq_path() -> str:
    """
    Resolve path to the FAQ JSONL file across common environments.
    """
    candidates = [
        "/content/shopunow_faqs.jsonl",
        os.path.join(os.getcwd(), "data", "shopunow_faqs.jsonl"),
        os.path.join(os.getcwd(), "shopunow_faqs.jsonl"),
    ]
    for c in candidates:
        if os.path.exists(c):
            return c
    raise FileNotFoundError("❌ shopunow_faqs.jsonl not found in common paths.")


def load_faq_documents(path: str) -> List[Document]:
    """
    Load and parse JSONL file into LangChain Documents.
    """
    docs = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                record = json.loads(line)
                question = record.get("question", "").strip()
                answer = record.get("answer", "").strip()
                dept = record.get("department", "unknown").strip()
                if not question or not answer:
                    continue
                combined_text = f"Q: {question}\nA: {answer}"
                docs.append(
                    Document(
                        page_content=combined_text,
                        metadata={
                            "department": dept,
                            "question": question,
                            "answer": answer,
                        },
                    )
                )
            except json.JSONDecodeError as e:
                print(f"⚠️ Skipping invalid JSON line: {e}")
    return docs


def build_faq_vector_store(docs: List[Document]) -> FAISS:
    """
    Build FAISS vector store with normalized embeddings.
    """
    if not docs:
        raise ValueError("No FAQ documents to build vector store.")
    embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
    dim = len(embedding_model.embed_query("hello world"))
    index = faiss.IndexFlatIP(dim)
    store = FAISS(
        embedding_function=embedding_model,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    ids = [f"faq_{i+1}" for i in range(len(docs))]
    store.add_documents(docs, ids=ids)
    return store


def get_faq_vector_store() -> Tuple[FAISS, List[Document]]:
    """
    Lazy initializer for the FAQ vector store.
    Safe for multi-threaded Gunicorn startup.
    """
    global _FAQ_VECTOR_STORE, _FAQ_DOCS, _FAQ_PATH

    if _FAQ_VECTOR_STORE is not None:
        return _FAQ_VECTOR_STORE, _FAQ_DOCS

    with _FAQ_INIT_LOCK:
        if _FAQ_VECTOR_STORE is not None:
            return _FAQ_VECTOR_STORE, _FAQ_DOCS
        try:
            _FAQ_PATH = resolve_faq_path()
            _FAQ_DOCS = load_faq_documents(_FAQ_PATH)
            _FAQ_VECTOR_STORE = build_faq_vector_store(_FAQ_DOCS)
            dept_set = {d.metadata.get("department", "unknown") for d in _FAQ_DOCS}
            print(
                f"✅ Built vector store with {len(_FAQ_DOCS)} FAQs across {len(dept_set)} departments",
                flush=True,
            )
        except Exception as e:
            print(f"❌ Error initializing FAQ store: {e}", flush=True)
            raise

    return _FAQ_VECTOR_STORE, _FAQ_DOCS


In [19]:
# =========================
# Cell B — Agent (lazy FAQ store, cosine/IP thresholds, escalation)
# =========================

import os
import json
import numpy as np
import random
from typing import Optional, List, Dict, Any, Literal, Tuple
from pydantic import BaseModel, Field
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from rapidfuzz import fuzz

# LangChain base
from langchain_core.documents import Document

# ✅ Import lazy FAQ loader (refactored Cell A.1)
try:
    from shopunow_faq import get_faq_vector_store  # if using separate module
except ImportError:
    from __main__ import get_faq_vector_store      # fallback for Colab single-file mode

# --------------------
# Determinism
# --------------------
random.seed(42)
np.random.seed(42)

# --------------------
# Lazy sentiment
# --------------------
_SENTIMENT = None
def detect_sentiment(text: str) -> Literal["negative", "neutral", "positive"]:
    global _SENTIMENT
    if _SENTIMENT is None:
        _SENTIMENT = SentimentIntensityAnalyzer()
    if not text:
        return "neutral"
    c = _SENTIMENT.polarity_scores(text).get("compound", 0.0)
    if c <= -0.3:
        return "negative"
    if c >= 0.3:
        return "positive"
    return "neutral"

# --------------------
# Department classifier
# --------------------
DEPT_KEYWORDS: Dict[str, List[str]] = {
    "Orders & Returns": [
        "order", "order status", "track order", "tracking", "shipment", "delivery",
        "package", "where is my order", "cancel order", "return", "refund",
        "replace", "exchange", "pickup"
    ],
    "Payments & Billing": [
        "payment", "upi", "card", "wallet", "cod", "invoice", "coupon", "billing",
        "charged", "charge", "emi", "price", "gst"
    ],
    "Customer Support": [
        "support", "contact", "help", "issue", "complaint", "agent",
        "human", "speak to", "phone", "call", "email", "hours", "timings"
    ],
    "HR & IT Helpdesk": [
        "password", "vpn", "access", "onboarding", "hardware", "software",
        "leave", "policy", "salary", "payroll", "payslip", "stipend",
        "salary date", "pay date", "salary delayed", "hrms", "hr portal"
    ],
}

def classify_department_with_confidence(user_query: str) -> Tuple[Optional[str], float, Dict[str, int]]:
    text = (user_query or "").lower()
    scores = {dept: sum(1 for kw in kws if kw in text) for dept, kws in DEPT_KEYWORDS.items()}
    top_dept, top_score = max(scores.items(), key=lambda x: x[1])
    if top_score == 0:
        return None, 0.0, scores
    # Ambiguity guard
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    if len(sorted_scores) > 1 and sorted_scores[1][1] == top_score:
        return None, 0.0, scores
    conf = 0.6 if top_score == 1 else (0.75 if top_score == 2 else 0.9)
    return top_dept, conf, scores

# --------------------
# Similarity thresholds
# --------------------
DEPT_SIM_THRESHOLDS = {
    "Orders & Returns": 0.80,
    "Payments & Billing": 0.78,
    "Customer Support": 0.75,
    "HR & IT Helpdesk": 0.80,
    None: 0.80,
}

# --------------------
# Agent State
# --------------------
class AgentState(BaseModel):
    user_input: str
    department: Optional[str] = None
    dept_confidence: float = 0.0
    sentiment: Optional[Literal["negative","neutral","positive"]] = None
    tools_used: List[str] = Field(default_factory=list)
    retrieved: List[Dict[str, Any]] = Field(default_factory=list)
    intent: Optional[Literal["rag","order_status","return_create","ticket","human_escalation","unknown"]] = None
    answer: Optional[str] = None
    confidence: float = 0.0
    reason: Optional[str] = None

# --------------------
# Helpers
# --------------------
def extract_answer_text(page_content: str) -> str:
    if not page_content:
        return page_content
    lc = page_content.lower()
    if "a:" in lc:
        idx = lc.find("a:")
        return page_content[idx + 2:].strip().lstrip(":").strip()
    if page_content.strip().startswith("Q:"):
        return page_content.replace("Q:", "", 1).strip()
    return page_content

def contains_any(text: str, keywords: List[str]) -> bool:
    low = text.lower()
    return any(kw in low for kw in keywords)

# --------------------
# Intent routing
# --------------------
def route_intent(state: AgentState) -> Dict[str, Any]:
    q = (state.user_input or "").lower()
    sentiment = detect_sentiment(state.user_input)
    dept, conf, _ = classify_department_with_confidence(state.user_input)
    if sentiment == "negative":
        intent = "human_escalation"
    elif contains_any(q, ["order status", "track order", "where is my order", "tracking", "shipment", "delivery", "package"]):
        intent = "order_status"
    elif contains_any(q, ["return", "refund", "replace", "exchange"]):
        intent = "rag" if contains_any(q, ["policy", "how many", "days", "window"]) else "return_create"
    elif contains_any(q, ["ticket", "helpdesk", "support issue", "complaint", "problem"]):
        intent = "ticket"
    else:
        intent = "rag"
    print(f"[route_intent] → intent={intent}, dept={dept}, conf={conf:.2f}, sentiment={sentiment}", flush=True)
    return {"intent": intent, "department": dept, "dept_confidence": conf, "sentiment": sentiment}

# --------------------
# Tool Node (lazy FAISS load)
# --------------------
def _filter_by_department(results, dept):
    if not results or not dept:
        return results or []
    filtered = [(d, s) for d, s in results if (d.metadata or {}).get("department") == dept]
    return filtered or results

def tool_node(state: AgentState) -> Dict[str, Any]:
    q = (state.user_input or "").strip()
    dept, conf, intent = state.department, state.dept_confidence, state.intent
    print(f"[tool_node] intent={intent}, dept={dept}, conf={conf:.2f}", flush=True)

    # direct intents
    if intent == "order_status":
        has_id = any(tok.startswith(("ORD-", "ord-")) or tok.isdigit() for tok in q.replace("#", " ").split())
        if not has_id:
            return {"answer": "Please share your Order ID (e.g. ORD-1234).", "tools_used": ["order_status_tool"], "confidence": 0.6}
        return {"answer": "Your order is being processed and will be shipped soon.", "tools_used": ["order_status_tool"], "confidence": 0.9}

    if intent == "return_create":
        return {"answer": "Return initiated. You’ll get pickup and label details by email.", "tools_used": ["return_create_tool"], "confidence": 0.9}

    if intent == "ticket":
        return {"answer": "A support ticket has been created. Our team will reach out shortly.", "tools_used": ["ticket_tool"], "confidence": 0.85}

    if intent == "human_escalation":
        return {"answer": "I’m sorry for the inconvenience. Escalating to human support.", "tools_used": ["escalation"], "confidence": 0.2}

    # rag retrieval
    if intent == "rag":
        if not dept or conf < 0.6:
            return {"answer": "This may relate to multiple areas. Escalating to human support.", "tools_used": ["escalation"], "confidence": 0.2}

        try:
            faq_store, faq_docs = get_faq_vector_store()
        except Exception as e:
            print(f"[tool_node] ❌ FAQ init failed: {e}", flush=True)
            return {"answer": "Error initializing knowledge base.", "tools_used": ["escalation"], "confidence": 0.0}

        try:
            results = faq_store.similarity_search_with_score(q, k=5)
            results = _filter_by_department(results, dept)
            if not results:
                return {"answer": "Sorry, I couldn’t find info. Escalating.", "tools_used": ["escalation"], "confidence": 0.2}
            doc, sim = results[0]
            th = DEPT_SIM_THRESHOLDS.get(dept, 0.8)
            if sim < th:
                best_doc, best_fuzzy = None, 0.0
                for d in faq_docs:
                    fs = fuzz.partial_ratio(q.lower(), d.metadata.get("question", "").lower()) / 100.0
                    if fs > best_fuzzy:
                        best_doc, best_fuzzy = d, fs
                if best_doc and best_fuzzy >= 0.92:
                    ans = extract_answer_text(best_doc.page_content)
                    dept_meta = best_doc.metadata.get("department", "unknown")
                    return {"answer": f"{ans} (Dept: {dept_meta})", "tools_used": ["rag_fuzzy_fallback"], "confidence": float(best_fuzzy)}
                return {"answer": "I’m not confident in my answer. Escalating.", "tools_used": ["escalation"], "confidence": float(sim)}
            ans = extract_answer_text(doc.page_content)
            dept_meta = doc.metadata.get("department", "unknown")
            return {"answer": f"{ans} (Dept: {dept_meta})", "tools_used": ["rag_retrieval"], "confidence": float(sim)}
        except Exception as e:
            print(f"[tool_node] ❌ Retrieval failed: {e}", flush=True)
            return {"answer": "Search error — escalating.", "tools_used": ["escalation"], "confidence": 0.0}

    # fallback
    return {"answer": "Could you please rephrase your question?", "tools_used": ["fallback"], "confidence": 0.3}

# --------------------
# Graph
# --------------------
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

graph = StateGraph(AgentState)
graph.add_node("route", route_intent)
graph.add_node("tool", tool_node)
graph.add_node("synth", lambda s: {})
graph.add_edge(START, "route")
graph.add_edge("route", "tool")
graph.add_edge("tool", "synth")
graph.add_edge("synth", END)

app = graph.compile(checkpointer=MemorySaver())

# --------------------
# ask()
# --------------------
def ask(user_query: str, thread_id: Optional[str] = None) -> str:
    import uuid
    if not thread_id:
        thread_id = f"thread_{uuid.uuid4().hex}"
    out = app.invoke({"user_input": user_query}, config={"configurable": {"thread_id": thread_id}})
    return out.get("answer", "No answer generated.")


In [20]:
# =========================
# Cell B.1 - Testing (Improved Output & Meaningful Names)
# =========================

import os
import uuid
from pprint import pprint

# Guard: only run this in dev / notebook environments, not in production (Render)
if os.getenv("RENDER_SERVICE_TYPE") or os.getenv("PORT"):
    print("[Skip] Testing cell running in production environment.")
else:
    test_queries = [
        "What are your support hours?",
        "Tell me order status for order id ORD-1234",
        "I want a return because the product is wrong",
        "My password reset isn't working, this is frustrating",
        "I submitted a complaint about a support issue",
        "How do I pay with UPI?",
        "How to apply for leaves?",
        "what is the leave policy?",
        "where is my order?",
        "my order is delayed",
        "whats the return polcy? how many days can i return the product?",
        "how many leaves can I take?",
        " how many days for rturn?",
        "my retturn is delayed",
        "Please replace my shirt size",
        "where is my order",
        "when will i get my salary",
        "my salaray is delayed",
        "I need help,",  # ambiguous
    ]

    for user_query in test_queries:
        conversation_id = f"conv_{uuid.uuid4().hex}"
        try:
            state = AgentState(user_input=user_query)
            route_info = route_intent(state)
            response = app.invoke(
                {"user_input": user_query},
                config={"configurable": {"thread_id": conversation_id}}
            )

            print("=" * 80)
            print(f"🧑 User Query: {user_query}")
            print(f"➡️ Intent: {route_info.get('intent')} | Dept: {route_info.get('department')} | Sentiment: {route_info.get('sentiment')}")
            print(f"🤖 Agent Answer: {response.get('answer', '⚠️ No answer')}")
            print(f"🛠️ Tools Used: {response.get('tools_used')}")
            if response.get("retrieved"):
                print("📄 Retrieved Context:")
                pprint(response["retrieved"])
            print("=" * 80 + "\n")
        except Exception as e:
            print(f"Error handling test query {user_query!r}: {e}")
            traceback.print_exc()


[route_intent] → intent=rag, dept=Customer Support, conf=0.75, sentiment=positive
[route_intent] → intent=rag, dept=Customer Support, conf=0.75, sentiment=positive
[tool_node] intent=rag, dept=Customer Support, conf=0.75
✅ Built vector store with 119 FAQs across 5 departments
🧑 User Query: What are your support hours?
➡️ Intent: rag | Dept: Customer Support | Sentiment: positive
🤖 Agent Answer: Yes, live chat is available from 9 AM to 9 PM IST on our website and mobile app. (Dept: Customer Support)
🛠️ Tools Used: ['rag_retrieval']

[route_intent] → intent=order_status, dept=Orders & Returns, conf=0.75, sentiment=neutral
[route_intent] → intent=order_status, dept=Orders & Returns, conf=0.75, sentiment=neutral
[tool_node] intent=order_status, dept=Orders & Returns, conf=0.75
🧑 User Query: Tell me order status for order id ORD-1234
➡️ Intent: order_status | Dept: Orders & Returns | Sentiment: neutral
🤖 Agent Answer: Your order is being processed and will be shipped soon.
🛠️ Tools Used: ['

In [33]:
# =========================
# Cell C — Flask API (Render + Colab Compatible, Fast Bind)
# =========================
import os
import sys
import traceback
import threading
import uuid
import socket
from flask import Flask, request, jsonify
from flask_cors import CORS

# --- Flask setup ---
flask_app = Flask(__name__)
CORS(flask_app)

def _debug(msg: str):
    """Flush-safe logger."""
    print(msg, flush=True)

# --------------------
# Agent bridge (lazy)
# --------------------
def call_agent(query: str) -> str:
    """Safely route query to any active agent."""
    try:
        if "ask" in globals() and callable(globals()["ask"]):
            _debug("[AGENT] Using ask()")
            return globals()["ask"](query)
        for name in ["agent_app", "graph_app", "app"]:
            obj = globals().get(name)
            if hasattr(obj, "invoke"):
                _debug(f"[AGENT] Using {name}.invoke()")
                out = obj.invoke(
                    {"user_input": query},
                    config={"configurable": {"thread_id": f"api-{uuid.uuid4().hex}"}}
                )
                return out.get("answer", "No answer generated.")
        return "⚠️ No active agent found."
    except Exception as e:
        _debug(f"[AGENT] ❌ Error in call_agent: {e}")
        traceback.print_exc(file=sys.stdout)
        return f"Internal error: {e}"

# --------------------
# Routes
# --------------------
@flask_app.route("/ask", methods=["POST", "GET"])
def ask_api():
    try:
        _debug("[API] /ask hit")
        query = ""
        if request.method == "POST":
            data = request.get_json(silent=True) or {}
            query = (data.get("query") or "").strip()
        else:
            query = (request.args.get("query") or "").strip()

        # If no query in GET, show a simple greeting or instructions
        if request.method == "GET" and not query:
            return jsonify({"message": "Welcome to ShopUNow API. Use /ask?query=hello or POST JSON {\"query\": ...}"}), 200

        if not query:
            return jsonify({"error": "Empty query"}), 400

        _debug(f"[API] Query: {query!r}")
        answer = call_agent(query)
        _debug(f"[API] Answer: {answer!r}")
        return jsonify({"query": query, "answer": answer})

    except Exception as e:
        _debug("[API] ❌ Exception:")
        traceback.print_exc(file=sys.stdout)
        return jsonify({"error": "Internal Server Error", "details": str(e)}), 500

@flask_app.route("/", methods=["GET"])
def home():
    return jsonify({"status": "ok", "message": "ShopUNow backend active"})

# --------------------
# Colab / Render detection + ngrok (dev) support
# --------------------
def _get_colab_secret(name: str):
    try:
        from google.colab import userdata
        return userdata.get(name)
    except Exception:
        return None

def _find_free_port():
    """Find a free port on localhost."""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        return s.getsockname()[1]

IS_RENDER = bool(os.getenv("RENDER_SERVICE_TYPE") or os.getenv("PORT"))
NGROK_AUTH_TOKEN = _get_colab_secret("NGROK_AUTH_TOKEN")

# ─────────────── Colab / Dev Mode ───────────────
if not IS_RENDER and NGROK_AUTH_TOKEN:
    PORT = _find_free_port()
    _debug(f"▶️ [Colab Mode] Starting Flask on port {PORT}")

    def _run_flask():
        flask_app.run(host="0.0.0.0", port=PORT, debug=False, use_reloader=False)
    threading.Thread(target=_run_flask, daemon=True).start()

    try:
        from pyngrok import ngrok
    except ImportError:
        import subprocess
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "pyngrok"], check=False)
        from pyngrok import ngrok

    try:
        _debug("🔑 Setting ngrok auth token")
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        ngrok.kill()
        tunnel = ngrok.connect(PORT, bind_tls=True)
        public_url = getattr(tunnel, "public_url", str(tunnel))
        _debug(f"🚀 Public URL: {public_url}")

        with open("/content/backend_url.txt", "w") as f:
            f.write(public_url.strip())
        _debug("📁 backend_url.txt written")

        print("\n✅ Test using this:")
        print(f'curl -X POST "{public_url}/ask" -H "Content-Type: application/json" -d \'{{"query":"Hello"}}\'')
    except Exception as e:
        _debug(f"❌ ngrok setup failed: {e}")
        traceback.print_exc(file=sys.stdout)

# ─────────────── Render / Production Mode ───────────────
else:
    PORT = int(os.getenv("PORT", 8000))
    _debug(f"▶️ [Render Mode] Starting Flask on 0.0.0.0:{PORT}")

    # Gunicorn will manage the server, so we only call run if __main__
    if __name__ == "__main__":
        _debug("🚀 Running Flask (dev fallback)")
        flask_app.run(host="0.0.0.0", port=PORT, debug=False)


▶️ [Colab Mode] Starting Flask on port 52265
🔑 Setting ngrok auth token * Serving Flask app '__main__'

 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:52265
 * Running on http://172.28.0.12:52265
INFO:werkzeug:Press CTRL+C to quit


🚀 Public URL: https://eecc914de7a2.ngrok-free.app
📁 backend_url.txt written

✅ Test using this:
curl -X POST "https://eecc914de7a2.ngrok-free.app/ask" -H "Content-Type: application/json" -d '{"query":"Hello"}'


In [34]:
# =========================
# Cell D — Streamlit Frontend (Colab + Render)
# =========================
import os
import sys
import subprocess
import threading
import time

IS_RENDER = bool(os.getenv("PORT") or os.getenv("RENDER_SERVICE_TYPE"))

def _debug(msg: str):
    print(msg, flush=True)

# ============================================================
# 🔹 COMMON STREAMLIT APP TEMPLATE
# ============================================================
def write_streamlit_app(default_api_url: str):
    app_code = f"""
import streamlit as st
import requests

st.set_page_config(page_title="ShopUNow Agent", layout="centered")
st.title("🛍️ ShopUNow AI Assistant")

api_url = st.sidebar.text_input("Backend URL", value="{default_api_url}")
st.sidebar.caption("URL to Flask /ask endpoint")

st.divider()
st.subheader("💬 Chat")

if "chat" not in st.session_state:
    st.session_state.chat = []

query = st.text_input("Ask something:")

if st.button("Ask"):
    if query.strip():
        st.session_state.chat.append(("🧑 You", query))
        try:
            resp = requests.post(api_url, json={{"query": query}}, timeout=25)
            if resp.status_code == 200:
                ans = resp.json().get("answer", "⚠️ No answer returned.")
            else:
                ans = f"⚠️ HTTP {{resp.status_code}}: {{resp.text}}"
        except Exception as e:
            ans = f"⚠️ Request failed: {{e}}"
        st.session_state.chat.append(("🤖 Agent", ans))

for sender, msg in st.session_state.chat:
    st.markdown(f"**{{sender}}:** {{msg}}")
"""
    with open("app_frontend.py", "w", encoding="utf-8") as f:
        f.write(app_code.strip())

# ============================================================
# 🔹 RENDER MODE (Production)
# ============================================================
if IS_RENDER:
    _debug("[Render] Setting up Streamlit frontend")

    try:
        import streamlit, requests
    except ImportError:
        deps = ["streamlit", "requests"]
        subprocess.run([sys.executable, "-m", "pip", "install", "-qU", *deps])
        import streamlit, requests

    # Prefer BACKEND_URL from environment; fallback to same host
    backend_url = os.getenv("BACKEND_URL", "").strip().rstrip("/")
    default_api_url = backend_url + "/ask" if backend_url else "/ask"

    write_streamlit_app(default_api_url)

    # On Render, gunicorn/startCommand will launch Streamlit:
    # streamlit run app_frontend.py --server.port $PORT
    # ✅ No background thread here — Render runs it automatically
    _debug(f"✅ Streamlit app ready. BACKEND_URL={default_api_url}")

# ============================================================
# 🔹 COLAB / DEV MODE (Auto-ngrok)
# ============================================================
else:
    _debug("[Colab/Dev] Launching Streamlit frontend...")

    try:
        import streamlit, requests
        from pyngrok import ngrok
    except ImportError:
        deps = ["streamlit", "requests", "pyngrok"]
        subprocess.run([sys.executable, "-m", "pip", "install", "-qU", *deps])
        import streamlit, requests
        from pyngrok import ngrok

    backend_url_file = "/content/backend_url.txt"
    default_api_url = "http://127.0.0.1:5000/ask"

    if os.path.exists(backend_url_file):
        try:
            with open(backend_url_file, "r") as f:
                url = f.read().strip()
            if url:
                default_api_url = url.rstrip("/") + "/ask"
                _debug(f"✅ Backend URL loaded: {default_api_url}")
        except Exception as e:
            _debug(f"⚠️ Could not read backend_url.txt: {e}")

    write_streamlit_app(default_api_url)

    # Launch Streamlit in background
    def run_frontend():
        _debug("▶️ Starting Streamlit on port 8501...")
        subprocess.run([
            sys.executable, "-m", "streamlit", "run", "app_frontend.py",
            "--server.port", "8501", "--server.headless", "true"
        ])

    threading.Thread(target=run_frontend, daemon=True).start()
    time.sleep(6)

    try:
        _debug("🌍 Opening ngrok tunnel for Streamlit (8501)...")
        tunnel = ngrok.connect(8501, bind_tls=True)
        public_url = getattr(tunnel, "public_url", str(tunnel))
        _debug(f"🚀 Streamlit frontend public URL: {public_url}")
    except Exception as e:
        _debug(f"⚠️ ngrok tunnel failed: {e}")


[Colab/Dev] Launching Streamlit frontend...
✅ Backend URL loaded: https://eecc914de7a2.ngrok-free.app/ask
▶️ Starting Streamlit on port 8501...
🌍 Opening ngrok tunnel for Streamlit (8501)...
🚀 Streamlit frontend public URL: https://83523456fd41.ngrok-free.app
